In [ ]:
import numpy as np
from typing import Callable, Tuple

In [ ]:
Q = np.diag([0.5, 1])

def f(x: np.ndarray) -> np.ndarray:
    tmp = (x - np.array([[1.0], [0]]))
    return 0.5 * tmp.T @ Q @ tmp

def df(x: np.ndarray) -> np.ndarray:
    return Q @ (x - np.array([[1.0], [0]]))

def ddf(x: np.ndarray) -> np.ndarray:
    return Q


In [ ]:
def c(x: np.ndarray) -> float:
    return x[0] ** 2 + 2 * x[0] - x[1]

def dc(x: np.ndarray) -> np.ndarray:
    return np.block([[2 * x[0] + 2, np.array([-1.0])]])


In [ ]:
def forward_diff(
        func: Callable[[np.ndarray], np.ndarray],
        x: np.ndarray) -> np.ndarray:
    
    eps = np.finfo(np.float64).eps
    eps = 1e-5

    x_copy = x.copy()
    x_copy[0] += eps
    d1 = (func(x_copy) - func(x)) / eps

    x_copy[0] -= eps
    x_copy[1] += eps
    d2 = (func(x_copy) - func(x)) / eps

    return np.block([[d1, d2]])


In [ ]:
def newtown_step(x0: np.ndarray, 
                 lambda0: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    H = ddf(x0) + forward_diff(lambda x: dc(x).T * lambda0, x0)
    C = dc(x0)
    tmp1 = -np.linalg.inv(np.block([[H, C.T], [C, np.array([0.0])]]))
    tmp2 = np.block([[df(x0) + C.T @ lambda0], [c(x0)]])
    dz = tmp1 @ tmp2
    dx = dz[0:2]
    dlambda = dz[2]
    return x0 + dx, lambda0 + dlambda

def newtown(x0: np.ndarray, 
            lambda0: np.ndarray, 
            step: int) -> Tuple[np.ndarray, np.ndarray]:
    for _ in range(step):
      x0, lambda0 = newtown_step(x0, lambda0)
    return x0, lambda0


In [ ]:
import matplotlib.pyplot as plt

xguess = np.array([[-3.0], [2.0]])
lambdaguess = np.array([[0.0]])
xnew, lambdanew = newtown(xguess, lambdaguess, 10)

plt.xlim((-4, 4))
plt.ylim((-4, 4))
xs = np.linspace(-4, 4)
plt.plot(xs, xs ** 2 + 2 * xs)
plt.scatter(xnew[0], xnew[1])
plt.scatter(xguess[0], xguess[1])
plt.grid(True)
plt.show()



In [ ]:
def gauss_newtown_step(x0: np.ndarray, 
                 lambda0: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    H = ddf(x0)
    C = dc(x0)
    tmp1 = -np.linalg.inv(np.block([[H, C.T], [C, np.array([0.0])]]))
    tmp2 =  np.block([[df(x0) + C.T @ lambda0], [c(x0)]])
    dz = tmp1 @ tmp2
    dx = dz[0:2]
    dlambda = dz[2]
    return x0 + dx, lambda0 + dlambda

def gauss_newtown(x0: np.ndarray, 
                  lambda0: np.ndarray, 
                  step: int) -> Tuple[np.ndarray, np.ndarray]:
    for _ in range(step):
      x0, lambda0 = gauss_newtown_step(x0, lambda0)
    return x0, lambda0


In [ ]:
xguess = np.array([[-3.0], [2.0]])
lambdaguess = np.array([[0.0]])
xnew, lambdanew = gauss_newtown(xguess, lambdaguess, 10)

plt.xlim((-4, 4))
plt.ylim((-4, 4))
xs = np.linspace(-4, 4)
plt.plot(xs, xs ** 2 + 2 * xs)
plt.scatter(xnew[0], xnew[1])
plt.scatter(xguess[0], xguess[1])
plt.grid(True)
plt.show()